# Fitting Data with Timelag ODE

The goal of this notebook is to describe adapting the timelag ODE (Mandel 2016) to run as a data interpolation tool for fuel moisture content. By adapting the covariance matrices of the Kalman Filter, the ODE can be made to closely reproduce known FMC values. The purpose of this is to use it as a physics-guided interpolation tool. The field study Carlson 2007 observed 1h FMC twice daily. We wish to have an hourly resolution dataset, so we will test transforming the observed carlson data into a 1h resolution dataset using the ODE to fill in the gaps.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from src.utils import read_yml, time_range, plot_styles
from src.models.moisture_ode import ODE_FMC, model_augmented, ext_kf

In [ ]:
# Cleaned data from Carlson study, run code to clean data from Vanderkamp to create this file
df1 = pd.read_excel("data/processed_data/ok_1h.xlsx")
df100 = pd.read_excel("data/processed_data/ok_100h.xlsx")
weather1 = pd.read_excel("data/processed_data/mesonet.xlsx")
weather2 = pd.read_excel("data/processed_data/dvdk_weather.xlsx")
ode_params = read_yml("etc/params_models.yaml", subkey="ode")

## Join Data

## Get test stretch of data

In [ ]:
df2 = df1.iloc[70:85].copy()
t0 = df2.date.min()
t1 = df2.date.max()

# Filter weather data to times
X = weather[
    (weather.date >= t0 - relativedelta(hours=1)) & (weather.date <= t1+relativedelta(hours=1))
]

print(f"Start Time: {t0}")
print(f"End Time: {t1}")
print(f"Total Times: {X.shape[0]}")
print(f"Number of Weather Observations: {X.shape[0]}")
print(f"Number of FMC Observations: {df2.shape}")

## Run ODE with periodic Data Assimilation

Rounding observed FM times to nearest hour for now just to build code for periodic assimilation.

Questions for Jan:
- Run ODE with minute time lag?
- OR, interpolate observed FMC data to the hour? Half hour with new data?

Analytic solution to simple ODE with single input $E$ constant over time:

$$
m_{t+\Delta t} = (1-e^{-1/T})E + e^{-1/T}m_t
$$

In [ ]:
times = X.date.to_numpy()
hours = len(times)     # total timesteps length

In [ ]:
# Rounding times to nearest hour for now
df2['date_rounded'] = df2['date'].dt.round('h')
df2[["date", "date_rounded"]].head()

In [ ]:
# Extend FM data with NAs to match weather data size
ref_idx = pd.DatetimeIndex(X['date'])
fm = (
    df2.set_index('date_rounded')
       .reindex(ref_idx)
       .rename_axis('date_rounded')
       .reset_index()
)

rh = X.rh.to_numpy()
temp = X.temp.to_numpy()
temp = temp + 273.15 # C to F
Ed = 0.924 * rh**0.679 + 0.000499 * np.exp(0.1 * rh) + 0.18 * (21.1 + 273.15 - temp) * (1 - np.exp(-0.115 * rh))
Ew = 0.618 * rh**0.753 + 0.000454 * np.exp(0.1 * rh) + 0.18 * (21.1 + 273.15 - temp) * (1 - np.exp(-0.115 * rh))
rain = X.precip.to_numpy()
d = fm["fm1"].to_numpy()

# Trim to first available data
ind0 = np.where(~np.isnan(d))[0].min()
Ed = Ed[ind0:]
Ew = Ew[ind0:]
rain = rain[ind0:]
d = d[ind0:]
times = times[ind0:]
hours = d.shape[0]

In [ ]:
# Check data
plt.scatter(np.arange(0, len(d)), d)

## Default Noise Variance

$R = 1e-03$

Setting initial $\Delta E=-2$

In [ ]:
# Background State and Covariance Matrices
u = np.zeros((2,hours))
u[:,0]=[d[0],-2.0]       # initialize,background state 
P = np.zeros((2,2,hours))
P[:,:,0] = np.array([[1e-3, 0.],
                  [0.,  1e-3]]) # background state covariance
Q = np.array([[1e-05, 0.],
            [0,  1e-03]]) # process noise covariance
H = np.array([[1., 0.]])  # first component observed
R = np.array([1e-3]) # data variance
T = 1.0 # timelag

In [ ]:
for t in range(1, hours):
    update_phase = ~np.isnan(d[t])
    if update_phase:
        u[:,t],P[:,:,t] = ext_kf(u[:,t-1],P[:,:,t-1],
                                  lambda uu: model_augmented(uu,Ed[t],Ew[t],rain[t],t, T=T),
                                  Q,d[t],H=H,R=R)
    else:
        u[:,t],P[:,:,t] = ext_kf(u[:,t-1],P[:,:,t-1],
                                  lambda uu: model_augmented(uu,Ed[t],Ew[t],rain[t],t, T=T),
                                  Q*0.0)

In [ ]:
plt.plot(times, Ed, **plot_styles["Ed"])
plt.plot(times, Ew, **plot_styles["Ew"])
plt.plot(times, rain, **plot_styles["rain"])
plt.xticks(rotation=90)
plt.scatter(times, d, **plot_styles["fm"])
plt.plot(times, u[0, :], **plot_styles["model"])
plt.legend()
plt.title(f"KF With {T=}, {R=}")

## Reduced Data Variance



In [ ]:
# Background State and Covariance Matrices
u = np.zeros((2,hours))
u[:,0]=[d[0],-2.0]       # initialize,background state 
P = np.zeros((2,2,hours))
P[:,:,0] = np.array([[1e-3, 0.],
                  [0.,  1e-3]]) # background state covariance
Q = np.array([[1e-05, 0.],
            [0,  1e-03]]) # process noise covariance
H = np.array([[1., 0.]])  # first component observed
R = np.array([1e-05]) # data variance
T = 1.0 # timelag

for t in range(1, hours):
    update_phase = ~np.isnan(d[t])
    if update_phase:
        u[:,t],P[:,:,t] = ext_kf(u[:,t-1],P[:,:,t-1],
                                  lambda uu: model_augmented(uu,Ed[t],Ew[t],rain[t],t, T=T),
                                  Q,d[t],H=H,R=R)
    else:
        u[:,t],P[:,:,t] = ext_kf(u[:,t-1],P[:,:,t-1],
                                  lambda uu: model_augmented(uu,Ed[t],Ew[t],rain[t],t, T=T),
                                  Q*0.0)

In [ ]:
plt.plot(times, Ed, **plot_styles["Ed"])
plt.plot(times, Ew, **plot_styles["Ew"])
plt.plot(times, rain, **plot_styles["rain"])
plt.xticks(rotation=90)
plt.scatter(times, d, **plot_styles["fm"])
plt.plot(times, u[0, :], **plot_styles["model"])
plt.legend()
plt.title(f"KF With {T=}, {R=}")

In [ ]:
# Background State and Covariance Matrices
u = np.zeros((2,hours))
u[:,0]=[d[0],-2.0]       # initialize,background state 
P = np.zeros((2,2,hours))
P[:,:,0] = np.array([[1e-3, 0.],
                  [0.,  1e-3]]) # background state covariance
Q = np.array([[1e-03, 0.],
            [0,  1e-03]]) # process noise covariance
H = np.array([[1., 0.]])  # first component observed
R = np.array([1e-5]) # data variance
T = 1 # timelag

for t in range(1, hours):
    update_phase = ~np.isnan(d[t])
    if update_phase:
        u[:,t],P[:,:,t] = ext_kf(u[:,t-1],P[:,:,t-1],
                                  lambda uu: model_augmented(uu,Ed[t],Ew[t],rain[t],t, T=T),
                                  Q,d[t],H=H,R=R)
    else:
        u[:,t],P[:,:,t] = ext_kf(u[:,t-1],P[:,:,t-1],
                                  lambda uu: model_augmented(uu,Ed[t],Ew[t],rain[t],t, T=T),
                                  Q*0.0)

plt.plot(times, Ed, **plot_styles["Ed"])
plt.plot(times, Ew, **plot_styles["Ew"])
plt.plot(times, rain, **plot_styles["rain"])
plt.xticks(rotation=90)
plt.scatter(times, d, **plot_styles["fm"])
plt.plot(times, u[0, :], **plot_styles["model"])
plt.legend()
plt.title(f"KF With {T=}, {R=}")

## Grid Search 

Vary $\Delta E$, $T$, $R$, and $Q$

In [ ]:
def ode_kf_fit(df, params, tstep = 1, fm_col = "fm1"):
    # Get data from df
    Ed = df.Ed
    Ew = df.Ew
    rain = df.rain
    d = df[fm_col]
    # times = df.date
    hours = df.shape[0]
    
    # Background State and Covariance Matrices
    sigmaR = params["data_variance"]
    sigmdaQ = params["process_variance"]
    T = params["T"] # timelag
    dE = params["deltaE"]
    
    # Initialize
    u = np.zeros((2,hours))
    u[:,0]=[d[0],dE]       # initialize,background state 
    P = np.zeros((2,2,hours))
    P[:,:,0] = np.array([[1e-3, 0.],
                      [0.,  1e-3]]) # background state covariance
    Q = np.array([[sigmdaQ, 0.],
                [0,  sigmdaQ]]) # process noise covariance
    H = np.array([[1., 0.]])  # first component observed
    R = np.array([sigmaR]) # data variance

    # Run
    for t in range(1, hours):
        update_phase = ~np.isnan(d[t])
        if update_phase:
            u[:,t],P[:,:,t] = ext_kf(u[:,t-1],P[:,:,t-1],
                                      lambda uu: model_augmented(uu,Ed[t],Ew[t],rain[t],t, T=T),
                                      Q,d[t],H=H,R=R)
        else:
            u[:,t],P[:,:,t] = ext_kf(u[:,t-1],P[:,:,t-1],
                                      lambda uu: model_augmented(uu,Ed[t],Ew[t],rain[t],t, T=T),
                                      Q*0.0)    

    return u

In [ ]:
df3 = pd.DataFrame({
    "Ed" : Ed,
    "Ew" : Ew,
    "rain" : rain,
    "fm1" : d
})

In [ ]:
ode_params

In [ ]:
ngrid = 4
Tgrid = np.linspace(0, 2, num=ngrid+1)[1:]
sRgrid = np.linspace(1e-6, 1e-1, num=ngrid)
sQgrid = np.linspace(1e-6, 1e-1, num=ngrid)
dEgrid = np.linspace(0, 5, num=ngrid)

In [ ]:
from itertools import product

param_combos = [
    (t, r, q, e)
    for t, r, q, e in product(Tgrid, sRgrid, sQgrid, dEgrid)
]   

In [ ]:
print(f"Unique Parameter Combos: {len(param_combos)}")

In [ ]:
results = {}
for i in range(0, len(param_combos)):
    t, r, q, e = param_combos[i]
    ode_params.update({
        "T": t,
        "data_variance": r,
        "process_variance": q,
        "deltaE": e
    })
    ui = ode_kf_fit(df3, ode_params, tstep = 1, fm_col = "fm1")
    results[i] = ui

In [ ]:
np.mean((df3.fm1 - results[0][0,:])**2)

In [ ]:
mse = [np.mean((df3.fm1 - results[r][0,:])**2) for r in results]
print(f"Min Error: {np.min(mse)}")
print(f"{np.argmin(mse)=}")
print(f"Min Param Combo: (T, R, Q, dE) = {param_combos[np.argmin(mse)]}")

In [ ]:
# Set all observations to na, run without KF
df3.loc[1:, "fm1"] = np.nan
results = {}
for i in range(0, len(param_combos)):
    t, r, q, e = param_combos[i]
    ode_params.update({
        "T": t,
        "data_variance": r,
        "process_variance": q,
        "deltaE": e
    })
    ui = ode_kf_fit(df3, ode_params, tstep = 1, fm_col = "fm1")
    results[i] = ui

mse = [np.nanmean((d - results[r][0,:])**2) for r in results]
print(f"Min Error: {np.min(mse)}")
print(f"{np.argmin(mse)=}")
print(f"Min Param Combo: (T, R, Q, dE) = {param_combos[np.argmin(mse)]}")

## Other Approaches

Starting with:
$$
m_t = (1-e^{-1/T})E + e^{-1/T}m_{t-1}
$$

Allow $E_t$ to vary over time, introduce $\Delta E$ so

$$
m_t = (1-e^{-1/T})[E_t + \Delta E] + e^{-1/T}m_{t-1}
$$

Let's plot the first two known points, then try this simple algorithm:
- Fix $T=1$
- Grid search $\Delta E$, plot solutions

In [ ]:
inds = np.where(~np.isnan(d))[0][0:2]
print(inds)
times = np.arange(inds[0], inds[1]+1)

plt.plot(times, Ed[times], **plot_styles["Ed"])
plt.plot(times, Ew[times], **plot_styles["Ew"])
plt.scatter(inds, d[inds], **plot_styles["fm"])
plt.grid()

In [ ]:
# Set up delta E grid, visualling choosing to go from -1 to -4
dE = np.linspace(-1, -4, 5)
print(f"{dE=}")

# Fixed Params
u = np.zeros((2,hours))
T = 1.0 # timelag

In [ ]:
u0=d[inds[0]]
u1=d[inds[1]+1]

In [ ]:
def run_ode(m0, deltaE, E):
    tsteps = E.shape[0]
    m = np.zeros(tsteps)
    m[0] = m0
    for t in range(1, tsteps):
        m[t] = (1-np.exp(-1))*(E[t]+deltaE) + np.exp(-1/T)*m[t-1]

    return m

In [ ]:
mods = []
for di in dE:
    mi = run_ode(m0=u0, deltaE = di, E = Ew[times])
    mods.append(mi)

mods = np.vstack(mods)

In [ ]:
plt.plot(times, Ed[times], **plot_styles["Ed"])
plt.plot(times, Ew[times], **plot_styles["Ew"])
plt.scatter(inds, d[inds], **plot_styles["fm"])
cmap = plt.get_cmap("Greens")  
colors = cmap(np.linspace(0, 1, len(mods)))  
for i, mi in enumerate(mods):
    plt.plot(times, mi, color=colors[i], label = fr"$\Delta$E = {dE[i]}")
plt.grid()
plt.legend()

## Fitting Params - Inverse Problem